In [1]:
# imports
# open data
# Fix beam search algorithm
# run beam search
# run results code

In [2]:
import os
# import sys
# import logging
# import datetime
import dill as pickle
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import OneHotEncoder
# import numpy as np
import pandas as pd

In [3]:
# # original py files
# sys.path.insert(1, 'methods')

from data_methods import getData, standardize
from dimensionality_reduction import reduce_dimensionality,reduce_with
from beamSearch import EMM, as_string
from adjPysubgroup import adjustedBestFirstSearch, adjustedDFS, adjustedApriori
from qualityMeasures import calc_result_bs, calc_result_ps
from interpretabilityMeasures import Feature_Correlation_Scores, DBI_beam, DBI_ps

# open all data

In [5]:
dct_datasets = {}

for dataset in ['Ionosphere','Mushroom','Adult','Soybean','Arrhythmia','Indoor']:

    with open(os.path.join(
        r'W:\OneDrive - TU Eindhoven\DS&AI\2024-2025\2024-2025 q1\2AMM20 - Research Topics in Data Mining\Research Project Phase\GitHub Code\Interpretable-Subgroup-Discovery-1\results_renamed',
        f'{dataset}-data-reductions.pkl'), 'rb') as f:
        data = pickle.load(f)

    dct_datasets[dataset] = data

## Fix Code

<b>Problem 1: </b> ```eta``` function generates all possible refinements of a seed by adding new conditions based on all features, regardless of whether a feature is already used in the seed; it does not prevent the addition of new conditions on attributes that are already used in the seed. Thus, seeds with a certain attribute can be refined by adding another condition that includes the same attribute.<br>
<b>Solution:</b> modify the ```eta``` function so that it is only possible to refine a seed using attributes that are not already present in the seed.


<b>Problem 2:</b> The current Beam Search algorithm does not check whether subgroups with similar conditions contain the same subgroups, e.g. attr1 <= 5 AND attr2 > 3 might select the same observations as attr1 <= 5 AND attr2 >= 4. This results in duplicate subgroups, defined by fairly similar rules. <br>
<b>Solution:</b> Check for subgroup equivalence before adding a new subgroup to the result set to verify whether it is unique in its observations
<br>

#### Finished code:
```python
Fixed code with comments
"""
The following code was adapted from W. Duivesteijn, T.C. van Dijk. (2021)
    Exceptional Gestalt Mining: Combining Magic Cards to Make Complex Coalitions Thrive. 
    In: Proceedings of the 8th Workshop on Machine Learning and Data Mining for Sports Analytics.
    Available from http://wwwis.win.tue.nl/~wouter/Publ/J05-EMM_DMKD.pdf
"""

# Package imports
import heapq
import numpy as np

# Classes
class BoundedPriorityQueue:
    """
    Used to store the <q> most promising subgroups
    Ensures uniqness
    Keeps a maximum size (throws away value with least quality)
    """

    def __init__(self, bound): 
        # Initializes empty queue with maximum length of <bound>
        self.values = []
        self.bound = bound
        self.entry_count = 0

    def add(self, element, quality, **adds): 
        # Adds <element> to the bounded priority queue if it is of sufficient quality
        new_entry = (quality, self.entry_count, element, adds)
        if (len(self.values) >= self.bound):
            heapq.heappushpop(self.values, new_entry)
        else:
            heapq.heappush(self.values, new_entry)

        self.entry_count += 1

    def get_values(self):
        # Returns elements in bounded priority queue in sorted order
        for (q, _, e, x) in sorted(self.values, reverse=True):
            yield (q, e, x)

    def show_contents(self):  
        # Prints contents of the bounded priority queue (used for debugging)
        print("show_contents")
        for (q, entry_count, e) in self.values:
            print(q, entry_count, e)

class Queue:
    """
    Used to store candidate solutions
    Ensures uniqness
    """

    def __init__(self): # Initializes empty queue
        self.items = []

    def is_empty(self): # Returns True if queue is empty, False otherwise
        return self.items == []

    def enqueue(self, item): # Adds <item> to queue if it is not already present
        if item not in self.items:
            self.items.insert(0, item)

    def dequeue(self): # Pulls one item from the queue
        return self.items.pop()

    def size(self): # Returns the number of items in the queue
        return len(self.items)

    def get_values(self): # Returns the queue (as a list)
        return self.items

    def add_all(self, iterable): # Adds all items in <iterable> to the queue, given they are not already present
        for item in iterable:
            self.enqueue(item)

    def clear(self): # Removes all items from the queue
        self.items.clear()
        
# Functions
def refine(desc, more):
    # Creates a copy of the seed <desc> and adds it to the new selector <more>
    # Used to prevent pointer issues with selectors
    copy = desc[:]
    copy.append(more)
    return copy

def as_string(desc):
    # Adds ' and ' to <desc> such that selectors are properly separated when the refine function is used
    return ' and '.join(desc)

def eta(seed, df, features, n_chunks = 5,prnt=False):
    # Returns a generator which includes all possible refinements of <seed> for the given <features> on dataset <df>
    # n_chunks refers to the number of possible splits we consider for numerical features
    if prnt:
        print("eta ", seed)

    # ! Find features present in the current seed and exclude them from the possible features:
    # ! -> function currently only checks whether the condition is already in there, allowing for different conditions on the same attributes
    used_features = set()
    for condition in seed:
        feature_name = condition.split()[0]
        used_features.add(feature_name)
    available_features = [f for f in features if f not in used_features]
    # ! end of adjusted code

    # only specify more on the elements that are still in the subset
    if seed != []:              
        d_str = as_string(seed)
        ind = df.eval(d_str)
        df_sub = df.loc[ind, ]
    else:
        df_sub = df

    for f in available_features:
        # if (df_sub[f].dtype == 'float64') or (df_sub[f].dtype == 'float32'): #get quantiles here instead of intervals for the case that data are very skewed
        if pd.api.types.is_numeric_dtype(df_sub[f]): # ! more reliable than comparing it to a float as .dtype returns an object and not a string
            column_data = df_sub[f]
            dat = np.sort(column_data)
            dat = dat[np.logical_not(np.isnan(dat))]
            for i in range(1,n_chunks+1): #determine the number of chunks you want to divide your data in
                x = np.percentile(dat, 100 * i / n_chunks) # ! changed from 100 / i which is INCORRECT
                candidate_leq = f"{f} <= {x}" # ! changed from .format to fstring
                # if not candidate in seed: # if not already there
                yield refine(seed, candidate_leq)
                candidate_gt = f"{f} > {x}"
                # if not candidate in seed: # if not already there
                yield refine(seed, candidate_gt)
        # elif (df_sub[f].dtype == 'object'):
        elif pd.api.types.is_categorical_dtype(df_sub[f]) or pd.api.types.is_object_dtype(df_sub[f]): # ! more reliable
            column_data = df_sub[f]
            uniq = column_data.dropna().unique()
            for i in uniq:
                candidate_eq = f"{f} == '{i}'"
                # if not candidate in seed: # if not already there
                yield refine(seed, candidate_eq)
                candidate_neq = f"{f} != '{i}'"
                # if not candidate in seed: # if not already there
                yield refine(seed, candidate_neq)
        # elif (df_sub[f].dtype == 'int64'):
        #     column_data = df_sub[f]
        #     dat = np.sort(column_data)
        #     dat = dat[np.logical_not(np.isnan(dat))]
        #     for i in range(1,n_chunks+1): #determine the number of chunks you want to divide your data in
        #         x = np.percentile(dat,100/i) #
        #         candidate = "{} <= {}".format(f, x)
        #         # if not candidate in seed: # if not already there
        #         yield refine(seed, candidate)
        #         candidate = "{} > {}".format(f, x)
        #         # if not candidate in seed: # if not already there
        #         yield refine(seed, candidate)
        # elif (df_sub[f].dtype == 'bool'):
        elif pd.api.types.is_bool_dtype(df_sub[f]): # ! more reliable
            # uniq = column_data.dropna().unique()
            for i in [True,False]: # ! instead of unique
                candidate_eq = f"{f} == '{i}'"
                # if not candidate in seed: # if not already there
                yield refine(seed, candidate_eq)
                candidate_neq = f"{f} != '{i}'"
                # if not candidate in seed: # if not already there
                yield refine(seed, candidate_neq)
        else:
            # assert False
            continue # ! skip for unsupported dtypes

def satisfies_all(desc, df, threshold=0.02):
    # Function used to check if subgroup with pattern <desc> is sufficiently big relative to its dataset <df>
    # A subgroup is sufficiently big if the proportion of data included in it exceeds <threshold>   
    d_str = as_string(desc)
    ind = df.eval(d_str)
    return sum(ind) >= len(df) * 0.02 

def eval_quality(desc, df, target):
    # Function used to calculate the solution's WRAcc
    sub_group = df[df.eval(as_string(desc))] 
    prop_p_sg = len(sub_group[sub_group[target]==1])/len(sub_group)
    prop_p_df = len(df[df[target]==1])/len(df)
    wracc = ((len(sub_group)/len(df))**1) * (prop_p_sg - prop_p_df) #for WRAcc a=1
    return wracc



def EMM(w, d, q, catch_all_description, df, target, n_chunks=5, ensure_diversity = False,prnt_level=True,prnt_seed=True,prnt_eta=False):
    """
    w - width of beam, i.e. the max number of results in the beam
    d - num levels, i.e. how many attributes are considered
    q - max results, i.e. max number of results output by the algorithm
    eta - a function that receives a description and returns all possible refinements
    satisfies_all - a function that receives a description and verifies wheather it satisfies some requirements as needed
    eval_quality - returns a quality for a given description. This should be comparable to qualities of other descriptions
    catch_all_description - the equivalent of True, or all, as that the whole dataset shall match
    df - dataframe of mined dataset
    features - features in scope
    target - column name of target attribute in df
    """
    features = [col for col in df.columns if col!='target']
    
    # Initialize variables
    resultSet = BoundedPriorityQueue(q) # Set of results, can contain results from multiple levels
    candidateQueue = Queue() # Set of candidate solutions to consider adding to the ResultSet
    candidateQueue.enqueue(catch_all_description) # Set of results on a particular level
    error = 0.00001 # Allowed error margin (due to floating point error) when comparing the quality of solutions

    # ! keep track of seen_subgroups:
    seen_subgroups = set()

    # Perform BeamSearch for <d> levels
    for level in range(d):
        if prnt_level:
            print("level : ", level)
        
        # Initialize this level's beam
        beam = BoundedPriorityQueue(w)

        # Go over all rules generated on previous level, or 'empty' rule if level = 0 
        for seed in candidateQueue.get_values():
            if prnt_seed:
                print("    seed : ", seed)
            
            # Start by evaluating the quality of the seed
            if seed != []:
                seed_quality = eval_quality(seed, df, target)
                beam.add(seed, seed_quality) # ! include seed itself in the beam
            else:
                seed_quality = float('-inf') # instead of 99

            # For all refinements created by eta function on descriptions (i.e features), which can be different types of columns
            # eta(seed) reads the dataset given certain seed (i.e. already created rules) and looks at new descriptions
            for desc in eta(seed, df, features, n_chunks,prnt=prnt_eta):

                # Check if the subgroup contains at least x% of data, proceed if yes
                if satisfies_all(desc, df):
                    # generate hash of subgroup indices
                    subgroup_indices = df[df.eval(as_string(desc))].index
                    subgroup_hash = tuple(sorted(subgroup_indices))

                    # skip if the (hash of the) subgroup has already been visited
                    if subgroup_hash in seen_subgroups:
                        continue
                    # if it has not been seen, we continue and add the new hash to the set
                    seen_subgroups.add(subgroup_hash)

                    # Calculate the new solution's quality
                    quality = eval_quality(desc, df, target)
                    
                    # Ensure diversity by forcing difference in quality when compared to its seed
                    # if <ensure_diversity> is set to True. Principle is based on:
                    # Van Leeuwen, M., & Knobbe, A. (2012), Diverse subgroup set discovery.
                    # Data Mining and Knowledge Discovery, 25(2), 208-242.
                    if ensure_diversity:
                        # if quality < (seed_quality * 1-error) or quality > (seed_quality * 1+error) : # ! <- irrelevantly long condition
                        if abs(quality - seed_quality) > error:
                            resultSet.add(desc, quality)
                            beam.add(desc, quality)
                    else:
                        resultSet.add(desc, quality)
                        beam.add(desc, quality)

        # When all candidates for a search level have been explored, 
        # the contents of the beam are moved into candidateQueue, to generate next level candidates
        candidateQueue = Queue()
        candidateQueue.add_all(desc for (_, desc, _) in beam.get_values())
        
    # Return the <resultSet> once the BeamSearch algorithm has completed
    return resultSet
```

## Run Beam Search algo's

In [19]:
df = dct_datasets['Soybean']['vanilla']
test = EMM(100, 3, 100, [], df, 'target', ensure_diversity=True)

level :  0
    seed :  []
level :  1
    seed :  ["attribute10 != '0'"]
    seed :  ["attribute14 == '1'"]
    seed :  ["attribute34 != '0'"]
    seed :  ["attribute2 != '0'"]
    seed :  ["attribute1 == '0'"]
    seed :  ["attribute28 != '1'"]
    seed :  ["attribute35 != '0'"]
    seed :  ["attribute7 == '2'"]
    seed :  ["attribute22 == '1'"]
    seed :  ["attribute1 == '3'"]
    seed :  ["attribute23 == '1'"]
    seed :  ["attribute24 == '1'"]
    seed :  ["attribute6 == '0'"]
    seed :  ["attribute1 == '6'"]
    seed :  ["attribute1 != '2'"]
    seed :  ["attribute6 == '2'"]
    seed :  ["attribute9 == '1'"]
    seed :  ["attribute10 != '2'"]
    seed :  ["attribute3 != '1'"]
    seed :  ["attribute17 != '0'"]
    seed :  ["attribute1 == '4'"]
    seed :  ["attribute1 != '1'"]
    seed :  ["attribute1 == '1'"]
    seed :  ["attribute1 != '4'"]
    seed :  ["attribute17 == '0'"]
    seed :  ["attribute3 == '1'"]
    seed :  ["attribute10 == '2'"]
    seed :  ["attribute35 != '2'"